# 模型存档与载入(Save and Loading)

## MNIST 手写阿拉伯数字辨识

In [1]:
import tensorflow as tf

mnist = tf.keras.datasets.mnist

# 载入 MNIST 手写阿拉伯数字资料
(x_train, y_train),(x_test, y_test) = mnist.load_data()

# 特征缩放，使用常态化(Normalization)，公式 = (x - min) / (max - min)
x_train_norm, x_test_norm = x_train / 255.0, x_test / 255.0

# 建立模型
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
])

# 设定优化器(optimizer)、损失函数(loss)、效能衡量指标(metrics)的类别
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# 模型训练
history = model.fit(x_train_norm, y_train, epochs=5, validation_split=0.2)

# 评分(Score Model)
score=model.evaluate(x_test_norm, y_test, verbose=0)

for i, x in enumerate(score):
    print(f'{model.metrics_names[i]}: {score[i]:.4f}')

Epoch 1/5
1500/1500 [==============================] - 5s 3ms/step - loss: 0.4529 - accuracy: 0.8667 - val_loss: 0.1329 - val_accuracy: 0.9622
Epoch 2/5
1500/1500 [==============================] - 2s 2ms/step - loss: 0.1339 - accuracy: 0.9605 - val_loss: 0.0999 - val_accuracy: 0.9700
Epoch 3/5
1500/1500 [==============================] - 2s 2ms/step - loss: 0.0913 - accuracy: 0.9732 - val_loss: 0.0885 - val_accuracy: 0.9732
Epoch 4/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.0680 - accuracy: 0.9787 - val_loss: 0.0808 - val_accuracy: 0.9751
Epoch 5/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.0548 - accuracy: 0.9823 - val_loss: 0.0843 - val_accuracy: 0.9747
loss: 0.0701
accuracy: 0.9782


## 模型存档(Save)

In [2]:
model.save('my_model')

INFO:tensorflow:Assets written to: my_model\assets


## 模型载入(Loading)

In [3]:
# 模型载入
model2 = tf.keras.models.load_model('my_model')

# 评分(Score Model)
score=model2.evaluate(x_test_norm, y_test, verbose=0)

for i, x in enumerate(score):
    print(f'{model2.metrics_names[i]}: {score[i]:.4f}')

loss: 0.0701
accuracy: 0.9782


In [4]:
# 模型比较
import numpy as np

# 比较，若结果不同，会出现错误
np.testing.assert_allclose(
    model.predict(x_test_norm), model2.predict(x_test_norm)
)

## Keras h5 模型存档(Save)

In [5]:
model.save('my_h5_model.h5')

In [6]:
# 模型载入
model3 = tf.keras.models.load_model('my_h5_model.h5')

## 取得模型结构

In [7]:
# 取得模型结构
config = model.get_config()

# 载入模型结构
# Sequential model
new_model = tf.keras.Sequential.from_config(config)

# function API
# new_model = tf.keras.Model.from_config(config)

## json 格式

In [8]:
# 取得模型结构
json_config = model.to_json()

# 载入模型结构
new_model = tf.keras.models.model_from_json(json_config)

## 取得模型权重

In [9]:
# 取得模型权重
weights = model.get_weights()
weights

[array([[-0.07456258,  0.06393789, -0.06428564, ..., -0.01037651,
         -0.00075018,  0.02070208],
        [-0.07116753, -0.0734145 ,  0.06361249, ...,  0.01945378,
          0.02640199, -0.01401142],
        [-0.07579142, -0.07441146,  0.01998896, ..., -0.00643145,
          0.04564028, -0.06786995],
        ...,
        [ 0.0488726 , -0.01030109,  0.06306989, ..., -0.06392061,
         -0.03373569, -0.05201003],
        [-0.04656494, -0.02243511, -0.03304058, ..., -0.06787682,
         -0.01773017,  0.07096189],
        [ 0.06254293,  0.01443324,  0.05800903, ...,  0.04117959,
         -0.05096102, -0.01709218]], dtype=float32),
 array([-0.09254983,  0.01622153, -0.0625335 ,  0.08666411,  0.10997227,
         0.12771368, -0.02746079, -0.0087847 ,  0.04165375, -0.03454573,
         0.03890204,  0.00739691, -0.01258934, -0.00979497, -0.09185535,
         0.02082837,  0.05723752,  0.10199914, -0.02323096,  0.04471245,
        -0.06626875, -0.09415329, -0.04521064,  0.0919965 ,  0.039

In [10]:
# 设定模型权重
new_model.set_weights(weights)

# 设定优化器(optimizer)、损失函数(loss)、效能衡量指标(metrics)的类别
new_model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# predict
score=new_model.evaluate(x_test_norm, y_test, verbose=0)
score

[0.07007579505443573, 0.9782000184059143]

## 取得模型结构时，Custom Layer 需注册

In [11]:
class CustomLayer(tf.keras.layers.Layer):
    def __init__(self, units=32, **kwargs):
        super(CustomLayer, self).__init__(**kwargs)
        self.units = units

    def build(self, input_shape):
        self.w = self.add_weight(
            shape=(input_shape[-1], self.units),
            initializer="random_normal",
            trainable=True,
        )
        self.b = self.add_weight(
            shape=(self.units,), initializer="random_normal", trainable=True
        )

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

    def get_config(self):
        config = super(CustomLayer, self).get_config()
        config.update({"units": self.units})
        return config


def custom_activation(x):
    return tf.nn.tanh(x) ** 2


# Make a model with the CustomLayer and custom_activation
inputs = tf.keras.Input((32,))
x = CustomLayer(32)(inputs)
outputs = tf.keras.layers.Activation(custom_activation)(x)
model = tf.keras.Model(inputs, outputs)

# Retrieve the config
config = model.get_config()

# Custom Layer 需注册
custom_objects = {"CustomLayer": CustomLayer, "custom_activation": custom_activation}
with tf.keras.utils.custom_object_scope(custom_objects):
    new_model = tf.keras.Model.from_config(config)

In [12]:
# 模型权重存档，有 Custom Layer 会出现错误
model.save_weights('my_h5_model.weight')

AttributeError: 'NoneType' object has no attribute 'replace'

In [ ]:
# 载入模型权重档
model.load_weights('my_h5_model.weight')